# Clean News 

# cleaning policies

- pl.col('authors').str.contains('Advertis Web')  
- 

In [2]:
import polars as pl
import pandas as pd

# From Raw News to Docs

## Select column

In [2]:
import polars as pl

In [9]:
filepath = '/home/sebacastillo/willow/output/news_narcotráfico_related_2023-08-09_1951.csv'
df = pl.read_csv(filepath)

In [4]:
df.columns

['date_extract', 'date_article', 'content', 'link', 'authors']

## Filter Row with Regex

In [92]:
# Positive
false_advertis = (
    pl.scan_csv(filepath)
    .filter(
        pl.col('authors').str.contains('Advertis Web')       
    )
    .select(
        [
            'link'
        ]
    )
).collect().to_pandas()

In [93]:
false_advertis

,link
0,https://www.elonce.com/secciones/policiales/76...
1,http://superdeportivo.elonce.com/secciones/pol...
2,https://www.diarioriouruguay.com.ar/policiales...


In [75]:
# Positive
false_clickaqui= (
    pl.scan_csv(filepath)
    .filter(
        pl.col('content').str.contains('Click aquí')       
    )
    .select(
        [
            'link',
            'content'
        ]
    )
).collect().to_pandas()
false_clickaqui['link'][0]

'https://www.elonce.com/secciones/policiales/766331-pact-una-cita-por-tinder-lo-drogaron-desvalijaron-y-est-grave.htm'

In [42]:
# Negative
q = (
    pl.scan_csv(filepath)
    .filter(
        pl.col('authors').str.contains('n-a').is_not()        
    )
    .select(
        [
            'authors'
        ]
    )
)
q.collect().to_series().value_counts()

authors,counts
str,u32
"""El Tucumano""",2
"""Ezequiel Miño""",1
"""Gustavo Berón-…",2
"""Daiana Larrat""",3
"""Guido Berrini""",3
"""Advertis Web T…",3
"""Redacción El""",2
"""Diario El Dia …",10
"""Florencia Ojed…",4


## Length of string

In [62]:
q = (
    pl.scan_csv(filepath)
    .select(
    [
        'content',
        pl.col('content').str.lengths().alias('content_length'),        
    ]
    )
).sort('content_length')
q.collect()

content,content_length
str,u32
"""Click aquí par…",342
"""Click aquí par…",342
"""Click aquí par…",342
"""📌La enfermera …",349
"""📌La enfermera …",349
"""La PFA realizó…",561
"""La PFA realizó…",561
"""Click aquí par…",587
"""Se trata del “…",865


## Split column

In [84]:
authors = (
    df
    .select(
    [
        'authors',
        pl.col('authors').str.split(' ').alias('splitname'),
    ])
)

In [86]:
authors

authors,splitname
str,list[str]
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""n-a""","[""n-a""]"
"""n-a""","[""n-a""]"
"""Gustavo Berón-…","[""Gustavo"", ""Berón-Candela"", … ""Lynch""]"
"""n-a""","[""n-a""]"


In [77]:
# Negative
(
    pl.scan_csv(filepath)
    .filter(
        pl.col('authors').str.contains('n-a').is_not()        
    )
    .select(
        [
            'authors'
        ]
    )
).collect().to_pandas()

,authors
0,Diario El Dia De La Plata Www.Eldia.Com-El Dia
1,Diario El Dia De La Plata Www.Eldia.Com-El Dia
2,Diario El Dia De La Plata Www.Eldia.Com-El Dia
3,Diario El Dia De La Plata Www.Eldia.Com-El Dia
4,Diario El Dia De La Plata Www.Eldia.Com-El Dia
...,...
90,El Día
91,El Día
92,Jorgelina Hernández
93,Daniel Quilodrán


## Aggregate

In [13]:
duplicados = (
    df
    .groupby("link", 'date_article')
    .agg(
        [
            pl.count().alias('count_ocurrence'),
            pl.col("content").str.n_chars().mean().alias("avg_content_length")
        ]
    )
    .sort("count_ocurrence", descending=True)
    .limit(10)
)

In [14]:
duplicados

link,date_article,count_ocurrence,avg_content_length
str,str,u32,f64
"""https://www.lm…","""2023-08-09T20:…",3,6087.666667
"""https://www.lm…","""2023-08-09T22:…",3,4983.666667
"""https://www.lm…","""2023-08-09T20:…",3,7583.0
"""https://www.el…","""2023-08-09T00:…",2,2772.0
"""https://www.el…","""2023-08-09T00:…",2,3554.0
"""https://www.el…","""2023-08-09T00:…",2,10787.0
"""https://www.el…","""2023-08-09T00:…",2,8065.0
"""https://www.el…","""2023-08-09T00:…",2,7480.0
"""https://www.el…","""2023-08-08T00:…",2,4370.0


### Transformation

In [105]:
df.columns

['date_extract', 'date_article', 'content', 'link', 'authors']

In [18]:
df = pl.read_csv('/home/sebacastillo/willow/output/news_narcotráfico_related_2023-08-09_1951.csv')
df = (
    df.with_columns([
        pl.col('date_extract').str.strptime(
            pl.Date,
            format='%Y-%m-%d %H:%M:%s', strict=True
        ),        

        pl.col('link').str.replace(r"https://www.", "").alias('new'),
        #pl.col('reference').str.replace_all(r"id-", ""),
        # ----------------
        #pl.col('reference').str.split(by=";")
        ##.struct.rename_fields(["first_part", "second_part"])
        ##.alias("fields"),
    ])
)
df[0]

date_extract,date_article,content,link,authors,new
date,str,str,str,str,str
2023-08-09,"""2023-08-09T00:…","""Estimado lecto…","""https://www.el…","""Diario El Dia …","""eldia.com/nota…"


In [19]:
isinstance(df, pl.DataFrame)

True

In [30]:
text = df['content'][0]

In [31]:
view_string(text)

['Estimado lector, muchas gracias por su interés en nuestras notas. Hemos incorporado el registro con ',
 'el objetivo de mejorar la información que le brindamos de acuerdo a sus intereses. Para más informac',
 'ión haga clic aquí Suplementos > Avisos > Servicios > Entretenimientos > Otros sitios > \n15 °C La Pl',
 'ata\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tMiércoles 9 de Agosto, 2023\n\t\t\t\t\t\t\t\t\t\t \n La muerte de una niña en La',
 'nús tras ser víctima de un ataque de motochorros atravesó la agenda electoral y llevó a los precandi',
 'datos bonaerenses de las principales fuerzas que competirán en las PASO del domingo próximo, Unión p',
 'or la Patria (UxP) y Juntos por el Cambio (JxC), a suspender sus actos de cierre de campaña previsto',
 's para hoy y mañana. Morena Domínguez, de 11 años, murió hoy luego de ser asaltada por motochorros, ',
 'que la atacaron para robarle sus pertenencias cuando caminaba hacia su colegio en la localidad de Vi',
 'lla Diamant